In [1]:
import pandas as pd

pathToGungorVict = '..//..//DS7004//u1720146_DS7004_courseworkCodeAndData//preparationWorks//fromDS7003_Gungor2018VictorianAuthorAttribution_NGram//Gungor_2018_VictorianAuthorAttribution_data-train.csv'
gungorVictRow = pd.read_csv(pathToGungorVict, encoding = 'ISO-8859-1')
#8 Charles Dickens 6914/ 14 George Eliot 2696/ 26 Jane Austen 4441

In [2]:
authorsLines = gungorVictRow.loc[gungorVictRow['author'] == 14] #the minimum: 2696 lines

In [3]:
authorsLines.shape

(2696, 2)

In [4]:
for i in [26, 8]:
    lines2696 = gungorVictRow.loc[gungorVictRow['author'] == i]
    lines2696 = lines2696.iloc[0:2696]
    authorsLines = authorsLines.append(lines2696)

In [5]:
pd.DataFrame(authorsLines.author.value_counts())

,author
26,2696
14,2696
8,2696


In [6]:
test = authorsLines.sample(frac=0.3, random_state=42)

train = authorsLines.drop(test.index)
train = train.sample(frac=1, random_state=42).reset_index()
test = test.reset_index()

In [7]:
print(test.index)

RangeIndex(start=0, stop=2426, step=1)


In [8]:
print(train.index)

RangeIndex(start=0, stop=5662, step=1)


In [9]:
test

,index,text,author
0,5503,deal of wine was drunk and everybody talked an...,8
1,15303,at and had more people o love than many a smal...,14
2,4672,was on fire and to be consulted in reference t...,8
3,26841,triumph and pursued her purpose careless of an...,26
4,4674,old house they live with us now they said i of...,8
...,...,...,...
2421,28076,mean â that he should have attached himself to...,26
2422,14468,make me do nothing you can t said bob yes i ca...,14
2423,16071,ht his less accurate grandmother poetry has al...,14
2424,27547,man might represent the county with such an es...,26


In [10]:
train

,index,text,author
0,5784,misfortune to have an uncle who died leaving h...,8
1,5327,this does you credit twenty eight returned the...,8
2,29275,time to avoid to hear her young companion excl...,26
3,15859,her an we begun to be frightened and could n t...,14
4,27851,a winding t believed herself at last within re...,26
...,...,...,...
5657,3426,she was always very secret in her ways let me ...,8
5658,5441,james who looked very for he was cleaning the ...,8
5659,5483,fire and upset the tea and make all sorts of n...,8
5660,5721,shame em with her good looks yet ha ha she ii ...,8


In [11]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [12]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
#nltk.download() no need to use this line after using it once(?)   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [13]:
# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [14]:
def parsing_sentence_set(text_df):
    sentences = []  # Initialize an empty list of sentences

    print("Parsing sentences from training set")
    for review in text_df["text"]:
        sentences += review_to_sentences(review, tokenizer)
    return sentences

In [15]:
authors_sentences = parsing_sentence_set(authorsLines)

Parsing sentences from training set


In [16]:
print(len(authors_sentences))

8088


In [17]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
#num_features = 300    # Word vector dimensionality                      
#min_word_count = 40   # Minimum word count                        
#num_workers = 4       # Number of threads to run in parallel
#context = 10          # Context window size                                                                                    
#downsampling = 1e-3   # Downsample setting for frequent words

num_features = 300    # Word vector dimensionality                      
min_word_count = 5    # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 5           # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words
epochs=15             #number of epochs

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
def form_model_from_sentences(sentences):
    print("Training model...")
    model = word2vec.Word2Vec(sentences, workers=num_workers, \
                size=num_features, min_count = min_word_count, \
                window = context, sample = downsampling, iter = epochs)

    # If you don't plan to train the model any further, calling 
    # init_sims will make the model much more memory-efficient.
    model.init_sims(replace=True)
    return model

In [18]:
model = form_model_from_sentences(authors_sentences)

2020-08-22 13:04:43,100 : INFO : collecting all words and their counts
2020-08-22 13:04:43,102 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


2020-08-22 13:04:44,371 : INFO : collected 9976 word types from a corpus of 8019053 raw words and 8088 sentences
2020-08-22 13:04:44,372 : INFO : Loading a fresh vocabulary
2020-08-22 13:04:44,509 : INFO : effective_min_count=5 retains 9956 unique words (99% of original 9976, drops 20)
2020-08-22 13:04:44,510 : INFO : effective_min_count=5 leaves 8018990 word corpus (99% of original 8019053, drops 63)
2020-08-22 13:04:44,537 : INFO : deleting the raw counts dictionary of 9976 items
2020-08-22 13:04:44,539 : INFO : sample=0.001 downsamples 55 most-common words
2020-08-22 13:04:44,540 : INFO : downsampling leaves estimated 5665155 word corpus (70.6% of prior 8018990)
2020-08-22 13:04:44,574 : INFO : estimated required memory for 9956 words and 300 dimensions: 28872400 bytes
2020-08-22 13:04:44,575 : INFO : resetting layer weights
2020-08-22 13:04:46,254 : INFO : training model with 4 workers on 9956 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2020-08-22 

2020-08-22 13:05:24,229 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-08-22 13:05:24,232 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-22 13:05:24,243 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-22 13:05:24,248 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-22 13:05:24,249 : INFO : EPOCH - 8 : training on 8019053 raw words (5665294 effective words) took 5.0s, 1140879 effective words/s
2020-08-22 13:05:25,255 : INFO : EPOCH 9 - PROGRESS: at 19.41% examples, 1091098 words/s, in_qsize 7, out_qsize 0
2020-08-22 13:05:26,272 : INFO : EPOCH 9 - PROGRESS: at 38.33% examples, 1073493 words/s, in_qsize 8, out_qsize 1
2020-08-22 13:05:27,272 : INFO : EPOCH 9 - PROGRESS: at 49.70% examples, 932576 words/s, in_qsize 7, out_qsize 0
2020-08-22 13:05:28,278 : INFO : EPOCH 9 - PROGRESS: at 61.70% examples, 868969 words/s, in_qsize 7, out_qsize 0
2020-08-22 13:05:29,287 : INFO : EPOCH 9 - P

2020-08-22 13:06:04,033 : INFO : precomputing L2-norms of word weight vectors


In [19]:
model.most_similar(positive=['king', 'woman'], negative=['man'])

<ipython-input-19-95bb2fc5eda1>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar(positive=['king', 'woman'], negative=['man'])


[('queen', 0.4187351167201996),
 ('earl', 0.40320053696632385),
 ('prince', 0.3588062822818756),
 ('conqueror', 0.34097394347190857),
 ('france', 0.3379788100719452),
 ('french', 0.33247894048690796),
 ('girl', 0.3150230050086975),
 ('throne', 0.3080468773841858),
 ('priests', 0.30735668540000916),
 ('richard', 0.30342161655426025)]

In [20]:
import gensim
all_stopwords = set(gensim.parsing.preprocessing.STOPWORDS)
all_stopwords

{'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amoungst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'bill',
 'both',
 'bottom',
 'but',
 'by',
 'call',
 'can',
 'cannot',
 'cant',
 'co',
 'computer',
 'con',
 'could',
 'couldnt',
 'cry',
 'de',
 'describe',
 'detail',
 'did',
 'didn',
 'do',
 'does',
 'doesn',
 'doing',
 'don',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eg',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'etc',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifte

In [21]:
#be careful: nword and counter must be integers --Chiu
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    index2word_set2 = all_stopwords
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set and word in index2word_set2: 
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    if nwords == 0:
        nwords = 1
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
        if counter%50 == 0:
            haha = counter; hihi = len(reviews)
            print(f"Review {haha} of {hihi}") #% (counter, len(reviews))
       # 
       # Call the function (defined above) that makes average feature vectors
        #reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
       #
       # Increment the counter
        counter = counter + 1
    return reviewFeatureVecs

In [22]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [23]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.
num_features = 300
clean_train_reviews = []
for review in train["text"]:
    #clean_train_reviews.append( review_to_wordlist( review, \
        #remove_stopwords=True ))
    clean_train_reviews.append( review_to_wordlist( review ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["text"]:
    #clean_test_reviews.append( review_to_wordlist( review, remove_stopwords=True ))
    clean_test_reviews.append( review_to_wordlist( review ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Review 0 of 5662
Review 50 of 5662
Review 100 of 5662


<ipython-input-21-8856e0a8f420>:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  featureVec = np.add(featureVec,model[word])


Review 150 of 5662
Review 200 of 5662
Review 250 of 5662
Review 300 of 5662
Review 350 of 5662
Review 400 of 5662
Review 450 of 5662
Review 500 of 5662
Review 550 of 5662
Review 600 of 5662
Review 650 of 5662
Review 700 of 5662
Review 750 of 5662
Review 800 of 5662
Review 850 of 5662
Review 900 of 5662
Review 950 of 5662
Review 1000 of 5662
Review 1050 of 5662
Review 1100 of 5662
Review 1150 of 5662
Review 1200 of 5662
Review 1250 of 5662
Review 1300 of 5662
Review 1350 of 5662
Review 1400 of 5662
Review 1450 of 5662
Review 1500 of 5662
Review 1550 of 5662
Review 1600 of 5662
Review 1650 of 5662
Review 1700 of 5662
Review 1750 of 5662
Review 1800 of 5662
Review 1850 of 5662
Review 1900 of 5662
Review 1950 of 5662
Review 2000 of 5662
Review 2050 of 5662
Review 2100 of 5662
Review 2150 of 5662
Review 2200 of 5662
Review 2250 of 5662
Review 2300 of 5662
Review 2350 of 5662
Review 2400 of 5662
Review 2450 of 5662
Review 2500 of 5662
Review 2550 of 5662
Review 2600 of 5662
Review 2650 of 56

In [24]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, train["author"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"true_author":test["author"], "pred_author":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


In [25]:
print(output)

      true_author  pred_author
0               8            8
1              14           14
2               8            8
3              26           26
4               8            8
...           ...          ...
2421           26           26
2422           14           14
2423           14           14
2424           26           26
2425           14           14

[2426 rows x 2 columns]


In [26]:
confusion_matrix = pd.crosstab(output['true_author'], output['pred_author'], rownames=['Actural'], colnames=['Predicted'])
print(confusion_matrix)

Predicted   8    14   26
Actural                 
8          714   66    9
14          76  717   55
26          20   32  737


In [27]:
import sklearn
sklearn.metrics.confusion_matrix(output['true_author'], output['pred_author'])

array([[714,  66,   9],
       [ 76, 717,  55],
       [ 20,  32, 737]], dtype=int64)

In [28]:
sklearn.metrics.precision_recall_fscore_support(output['true_author'], output['pred_author'], average=None)

(array([0.88148148, 0.8797546 , 0.92009988]),
 array([0.90494297, 0.84551887, 0.93409379]),
 array([0.89305816, 0.86229705, 0.92704403]),
 array([789, 848, 789], dtype=int64))

In [29]:
pd.DataFrame(output.true_author.value_counts())

,true_author
14,848
26,789
8,789


In [33]:
word_vectors = model.wv.syn0

<ipython-input-33-98a1d4cbf9c6>:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  word_vectors = model.wv.syn0


In [34]:
word_vectors.shape

(9956, 300)

In [48]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.syn0
num_clusters = word_vectors.shape[0] / 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = int(num_clusters) )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print("Time taken for K Means clustering: ", elapsed, "seconds.")

<ipython-input-48-c310dcc16c02>:8: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  word_vectors = model.wv.syn0


Time taken for K Means clustering:  496.34906005859375 seconds.


In [49]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [69]:
import operator

In [72]:
max(word_centroid_map, key=word_centroid_map.get)

'descriptive'

In [74]:
word_centroid_map.get('descriptive')

1990

In [50]:
word_centroid_map, len(word_centroid_map.keys())

({'the': 1608,
  'and': 289,
  'to': 765,
  'of': 730,
  'a': 413,
  'i': 910,
  'in': 391,
  'was': 1310,
  'that': 443,
  'her': 165,
  'it': 962,
  'he': 962,
  'as': 1374,
  'you': 910,
  'she': 962,
  'his': 277,
  'with': 1716,
  'had': 780,
  'not': 671,
  'for': 833,
  'be': 808,
  's': 154,
  'but': 587,
  'at': 1441,
  'on': 391,
  'have': 78,
  'is': 1015,
  'my': 154,
  'by': 405,
  'him': 865,
  'said': 1475,
  'me': 865,
  'which': 1092,
  'mr': 594,
  'all': 974,
  'so': 1374,
  'this': 651,
  'no': 464,
  'from': 787,
  'been': 1271,
  'were': 662,
  'would': 1385,
  'if': 1130,
  'an': 1690,
  'there': 1593,
  'what': 1632,
  'when': 289,
  't': 239,
  'they': 317,
  'could': 1385,
  'or': 934,
  'very': 191,
  'one': 1960,
  'more': 547,
  'we': 61,
  'who': 1223,
  'do': 401,
  'any': 464,
  'will': 265,
  'mrs': 594,
  'are': 1936,
  'them': 482,
  'out': 156,
  'than': 934,
  'their': 1342,
  'little': 681,
  'your': 154,
  'now': 910,
  'up': 1008,
  'some': 1798,

In [53]:
type(word_centroid_map)

dict

In [54]:
# For the first 10 clusters
for cluster in range(0,10):
    #
    # Print the cluster number  
    #print "\nCluster %d" #% cluster
    print(f"\nCluster {cluster}")
    #
    # Find all of the words for that cluster number, and print them out
    a_view = word_centroid_map.items()
    tuples = list(a_view)
    words = []
    for i in range(0,len(word_centroid_map.values())):
        if( tuples[i][1] == cluster ):
            words.append(tuples[i][0])
    print(words)


Cluster 0
['behind', 'stained', 'panes']

Cluster 1
['liking', 'preference']

Cluster 2
['cure', 'tion', 'bearing', 'doctrine', 'tendency', 'prophecy', 'species', 'gradual', 'development', 'bias', 'utility', 'consumption', 'import', 'diet', 'verge', 'analysis']

Cluster 3
['ice', 'blown', 'ocean', 'crooked', 'flies', 'sails', 'watery', 'brood', 'weed', 'isle', 'climb', 'honey', 'fuel', 'forests', 'gems', 'vegetation', 'shoots', 'multitudes', 'tides', 'sheds']

Cluster 4
['marked', 'coarse', 'polished', 'resembled', 'dissatisfied', 'adorned', 'moves', 'delicately', 'contrasted', 'disordered', 'gifted', 'richly', 'imaginable', 'settles']

Cluster 5
['v', 'x', 'iv', 'vi', 'vm', 'vl', 'vn', 'thk', 'problems']

Cluster 6
['fatal', 'decisive', 'speedy']

Cluster 7
['winked', 'complacently']

Cluster 8
['appointed', 'repaired', 'reappeared']

Cluster 9
['disturb', 'detain']


In [55]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [61]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["text"].size, int(num_clusters)), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros((test["text"].size, int(num_clusters)), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

In [59]:
type(train)

pandas.core.frame.DataFrame

In [60]:
train.iloc[0]

index                                                  5784
text      misfortune to have an uncle who died leaving h...
author                                                    8
Name: 0, dtype: object

In [64]:
# This cell take some minutes
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train["author"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"true_author":test["author"], "pred_author":result})
output.to_csv( "BagOfCentroidsAuthor.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


In [65]:
output

,true_author,pred_author
0,8,8
1,14,14
2,8,8
3,26,26
4,8,8
...,...,...
2421,26,26
2422,14,14
2423,14,14
2424,26,26


In [66]:
confusion_matrix = pd.crosstab(output['true_author'], output['pred_author'], rownames=['Actural'], colnames=['Predicted'])
print(confusion_matrix)

Predicted   8    14   26
Actural                 
8          756   28    5
14           8  834    6
26           2    3  784


In [67]:
import sklearn
sklearn.metrics.confusion_matrix(output['true_author'], output['pred_author'])

array([[756,  28,   5],
       [  8, 834,   6],
       [  2,   3, 784]], dtype=int64)

In [68]:
sklearn.metrics.precision_recall_fscore_support(output['true_author'], output['pred_author'], average=None)

(array([0.98694517, 0.96416185, 0.98616352]),
 array([0.9581749 , 0.98349057, 0.99366286]),
 array([0.97234727, 0.9737303 , 0.98989899]),
 array([789, 848, 789], dtype=int64))